In [18]:
# First we need to mount the Google drive
import os
# from google.colab import drive
# drive.mount('/content/Hadrive')

In [19]:

configs = dict({
"1": { "lr_initial": 3e-2, "decay": 0.9},
"2": { "lr_initial": 1e-2, "decay": 0.9}
})


for index, config in configs.items():
  print(f"index: {index}")
  print(f"config: {config}")
  for key,value in config.items():
    print(f"key: {key}, value: {value}")



index: 1
config: {'lr_initial': 0.03, 'decay': 0.9}
key: lr_initial, value: 0.03
key: decay, value: 0.9
index: 2
config: {'lr_initial': 0.01, 'decay': 0.9}
key: lr_initial, value: 0.01
key: decay, value: 0.9


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [21]:

data_path = '../../../data'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transform)

cifar10_val = datasets.CIFAR10(data_path, train=False, download=False, transform=transform)

# cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transform)

# cifar10_val = datasets.CIFAR10(data_path, train=False, download=False, transform=transform)

In [22]:
isinstance(cifar10, torch.utils.data.Dataset)

True

In [23]:
len(cifar10)

50000

In [24]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                           shuffle=True)

In [25]:
# model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        # bài toán phân loại 10 lớp nên output ra 10 nodes
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        # flatten về dạng vector để cho vào neural network
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


In [26]:
import pickle

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, data_path):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        correct = 0
        with torch.no_grad():
            for data in val_loader:
                images, labels = data
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                c = (predicted == labels).squeeze()
                correct += c.sum()
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(cifar10_val)))
            #save the model config
            model_state = model.state_dict()
            optimizer_state = optimizer.state_dict()
            dict_state = dict()
            dict_state["epoch"] = epoch
            dict_state["model_state"] = model_state
            dict_state["optimizer_state"] = optimizer_state

            try:
                geeky_file = open( "./epoch_" + str(epoch), 'wb')
                pickle.dump(dict_state, geeky_file)
                geeky_file.close()

            except:
                print("Something went wrong")

In [27]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=3e-2)
loss_fn = nn.CrossEntropyLoss()

In [28]:
for index, config in configs.items():
  print(f"index: {index}")
  # for key,value in config.items():
  #   print(f"key: {key}, value: {value}")
  data_path_index = data_path + "config_" + str(index) + "_"
  model = Net()
  optimizer = optim.SGD(model.parameters(), lr=config["lr_initial"])
  loss_fn = nn.CrossEntropyLoss()

  training_loop(
      n_epochs = 2,
      optimizer = optimizer,
      model = model,
      loss_fn = loss_fn,
      train_loader = train_loader,
      val_loader = val_loader,
      data_path = data_path_index
  )

index: 1
Epoch 1, Training loss 1.8269249892905546, Val accuracy 0.4327000081539154
Epoch 2, Training loss 1.4776185061925513, Val accuracy 0.4499000012874603
index: 2
Epoch 1, Training loss 2.041237510073825, Val accuracy 0.3544999957084656
Epoch 2, Training loss 1.7876445643432306, Val accuracy 0.40619999170303345


In [29]:
import pandas as pd
epoch = 1
path = "./epoch_" + str(epoch)
obj = pd.read_pickle(path)
print(obj.keys())

dict_keys(['epoch', 'model_state', 'optimizer_state'])
